# Vitessce Widget Tutorial

# Visualization of ComparativeData object

In [ ]:
# dask version: 2023.8.0

In [ ]:
# git checkout keller-mark/uv-and-pagemode
# source .venv/bin/activate
# uv sync --extra all
# uv pip install -e "../compasce"
# uv run jupyter lab

In [ ]:
#!uv pip install -e "../../../compasce"

In [ ]:
!pwd

In [ ]:
from compasce import run_all, create_dask_client
from anndata import read_h5ad
from os.path import join

In [ ]:
DATA_DIR = join("..", "..", "..", "compasce")

In [ ]:
def get_adata():
    adata = read_h5ad(join(DATA_DIR, "KPMP_PREMIERE_SC_version1.5_ForExplorer_withRC.032624.h5ad"))
    # Restrict to a subset of data when in the notebook, for now.
    adata = adata[0:25_000, 0:20_000].copy()
    adata.layers["counts"] = adata.raw[0:25_000, 0:20_000].X.todense()
    adata.raw = None
    return adata

In [ ]:
sample_id_col = "SampleID"
sample_group_pairs = [
    [
      ['diseasetype', 'AKI'],
      ['diseasetype', 'Reference'],
    ],
    [
      ['diseasetype', 'AKI'],
      ['diseasetype', 'CKD'],
    ],
    [
      ['diseasetype', 'CKD'],
      ['diseasetype', 'Reference'],
    ],
]

In [ ]:
run_all(
    get_adata,
    zarr_path=join(DATA_DIR, "kpmp_premiere.cdata.zarr"),
    client=create_dask_client(memory_limit='2GB'),
    sample_id_col=sample_id_col,
    sample_group_pairs=sample_group_pairs,
)

## Configure Vitessce

In [ ]:
#!uv pip install esbuild_py

In [12]:
from vitessce import (
    VitessceConfig,
    Component as cm,
    CoordinationType as ct,
    AnnDataWrapper,
    CsvWrapper,
)
from esbuild_py import transform

## Configure the data and views

In [13]:
vc = VitessceConfig(schema_version="1.0.17", name='ComparativeData example')
dataset = vc.add_dataset(name='KPMP Atlas Data').add_object(
    AnnDataWrapper(
        adata_url="https://storage.googleapis.com/vitessce-demo-data/scmd-analysis-october-2023/lake_et_al.2.h5ad.zarr",
        obs_feature_matrix_path='layers/normalize_pearson_residuals',
        obs_embedding_paths=['obsm/X_densmap'],
        obs_embedding_names=['densMAP'],
        obs_set_paths=['obs/cell_type'],
        obs_set_names=['Cell Type'],
        feature_labels_path='var/feature_name',
        sample_edges_path='obs/donor_id', # TODO: does this need to be implemented in the wrapper class?
        coordination_values={
          "obsType": 'cell',
          "featureType": 'gene',
          "featureValueType": 'expression',
          "sampleType": 'sample',
        }
    )
).add_object(
    CsvWrapper(
        csv_url="https://storage.googleapis.com/vitessce-demo-data/scmd-analysis-october-2023/20231129_OpenAccessClinicalData.csv",
        data_type="sampleSets",
        options={
          "sampleIndex": 'Participant ID',
          "sampleSets": [
            {
              "name": 'Tissue Type',
              "column": 'Tissue Type',
            },
            {
              "name": 'Hypertension',
              "column": 'Hypertension',
            },
          ],
        },
        coordination_values={
          "sampleType": 'sample',
        },
    )
)
biomarker_select_view = vc.add_view("biomarkerSelect", dataset=dataset, uid="biomarker-select").set_props(
    stratificationOptions=[
      {
        "stratificationId": 'aki-vs-hr',
        "name": 'Acute kidney injury (AKI) vs. Healthy reference',
        "stratificationType": 'sampleSet', #// key changed from 'groupType'. value changed from 'clinical'
        "sampleSets": [
          ['Tissue Type', 'AKI'],
          ['Tissue Type', 'Healthy Reference'],
        ],
      },
      {
        "stratificationId": 'aki-vs-hckd',
        "name": 'Acute kidney injury (AKI) vs. Chronic kidney disease attributed to hypertension (H-CKD)',
        "stratificationType": 'sampleSet',
        "sampleSets": [
          ['Tissue Type', 'AKI'],
          ['Tissue Type', 'CKD'],
        ],
      },
      {
        "stratificationId": 'dckd-vs-hr',
        "name": 'Chronic kidney disease attributed to diabetes (D-CKD) vs. Healthy reference',
        "stratificationType": 'sampleSet',
        "sampleSets": [
          ['Tissue Type', 'CKD'],
          ['Tissue Type', 'Healthy Reference'],
        ],
      },
    ]
)
scatterplot = vc.add_view('scatterplot', dataset=dataset, uid='scatterplot-case').set_props(title='CKD')
scatterplot2 = vc.add_view('scatterplot', dataset=dataset, uid='scatterplot-control').set_props(title='Healthy Reference')
obsSets = vc.add_view('obsSets', dataset=dataset, uid='cell-sets')
obsSetSizes = vc.add_view('obsSetSizes', dataset=dataset)
featureList = vc.add_view('featureList', dataset=dataset)
violinPlots = vc.add_view('obsSetFeatureValueDistribution', dataset=dataset, uid='violin-plot')
dotPlot = vc.add_view('dotPlot', dataset=dataset, uid='dot-plot')

sampleSetScope_case, sampleSetScope_control, sampleSetScope_caseControl = vc.add_coordination("sampleSetSelection", "sampleSetSelection", "sampleSetSelection")
sampleSetScope_case.set_value([['Tissue Type', 'CKD']])
sampleSetScope_control.set_value([['Tissue Type', 'Healthy Reference']])
sampleSetScope_caseControl.set_value([['Tissue Type', 'CKD'], ['Tissue Type', 'Healthy Reference']])

vc.link_views_by_dict([scatterplot], {
    "embeddingType": 'densMAP',
    "embeddingContoursVisible": True,
    "embeddingPointsVisible": False,
    "sampleType": 'sample',
    "sampleSetSelection": sampleSetScope_case,
}, meta=False)
vc.link_views_by_dict([scatterplot2], {
    "embeddingType": 'densMAP',
    "embeddingContoursVisible": True,
    "embeddingPointsVisible": False,
    "sampleType": 'sample',
    "sampleSetSelection": sampleSetScope_control,
}, meta=False)

vc.link_views([obsSets, obsSetSizes, featureList, violinPlots, dotPlot], ['sampleType'], ['sample']);
vc.link_views_by_dict([obsSets, obsSetSizes, featureList, violinPlots, dotPlot], {
    "sampleSetSelection": sampleSetScope_caseControl,
}, meta=False)
vc.link_views_by_dict([scatterplot, scatterplot2, violinPlots, featureList, dotPlot], {
    "featureSelection": ['ENSG00000169344'], # // , 'ENSG00000074803', 'ENSG00000164825'],
    "obsColorEncoding": 'geneSelection',
    "featureValueColormapRange": [0, 0.25],
}, meta=False)

vc.link_views_by_dict([scatterplot, scatterplot2], {
    "embeddingZoom": None,
    "embeddingTargetX": None,
    "embeddingTargetY": None,
}, meta=False)

In [14]:
# vc.to_dict(base_url="")

## Define the page layout

In [15]:
# Reference: https://github.com/vitessce/vitessce/blob/main/examples/configs/src/view-configs/lake-2023.js
PAGE_ESM = transform("""
function createPage(utilsForPages) {
  const {
    React,
    usePageModeView,
  } = utilsForPages;
  function PageComponent(props) {
    const BiomarkerSelect = usePageModeView('biomarker-select');
    const ScatterplotCase = usePageModeView('scatterplot-case');
    const ScatterplotControl = usePageModeView('scatterplot-control');
    const CellSets = usePageModeView('cell-sets');
    const ViolinPlot = usePageModeView('violin-plot');
    const DotPlot = usePageModeView('dot-plot');
    
    return (
        <>
          <style>{`
          h1, h2, h3, h4, h5, h6 {
            font-family: sans-serif;
          }
          h1 {
            font-weight: normal;
          }
          h2 {
            font-size: 36px;
          }
          h3 {
            font-size: 28px;
          }
          `}
          </style>
          <div style={{ width: '100%' }}>
              <h1>Comparative visualization of single-cell atlas data</h1>
              <BiomarkerSelect />
          </div>

          <div style={{ width: '100%', display: 'flex', flexDirection: 'row' }}>
            <div style={{ width: '100%'}}>
              <div style={{ width: '100%', display: 'flex', flexDirection: 'row' }}>
                <div style={{ width: '45%' }}><h2>Chronic Kidney Disease</h2></div>
                <div style={{ width: '5%' }}><h2 style={{ textAlign: 'right' }}>vs.&nbsp;</h2></div>
                <div style={{ width: '50%' }}><h2>Healthy Reference</h2></div>
              </div>
              <h3>Cell type-level representations</h3>
              <div style={{ width: '100%', height: '800px', display: 'flex', flexDirection: 'row' }}>
                <div style={{ width: '50%' }}>
                  <ScatterplotCase />
                </div>
                <div style={{ width: '50%' }}>
                  <ScatterplotControl />
                </div>
              </div>
              <div style={{ width: '100%', height: '500px' }}>
                <ViolinPlot />
              </div>
              <div style={{ width: '100%', height: '500px' }}>
                <DotPlot />
              </div>
            </div>
            <div style={{ width: '14%', height: '500px', marginTop: '213px' }}>
              <CellSets />
            </div>
          </div>

        </>
      );
  }
  return PageComponent;
}
export default { createPage };
""")

## Render page as widget

In [16]:
vw = vc.widget(js_package_version='3.5.1', page_esm=PAGE_ESM, page_mode=True, height=3000)
vw

VitessceWidget(config={'version': '1.0.17', 'name': 'ComparativeData example', 'description': '', 'datasets': …